
## Машинний аналіз тексту. Популярні алгоритми 2016


* Старий підхід -  [мішок зі словами (bag of words)](https://en.wikipedia.org/wiki/Bag-of-words_model) - не враховує граматику і навіть порядок слів
* З 2013 року вибух популярності іншого підходу - "вектори слів" (word embeddings). Алгоритми: [](word2vec)
* Кожне слово стає багатовимірним вектором (розмірності 50-1000) у лінійному просторі. Алгоритм враховує лексичний контекст (оточення слова). Що це дає? (всі векторні операції)
* Що це дає, нормальною мовою?
 * синоніми
 * яке слово зайве?
 * пошук схожих текстів (у яких використані різні слова)
 * аналогії (hello word, для word2vec)
 * дослідження еволюції концептів у мові
 * кластеризація, класифікація
 * переклад
 * карта найуживаніших слів 
 * приклад StitchFix



***
### Інші цікаві алгоритми

* LDA - алгоритм для автоматичного визначення тем у текстах ([лінк](https://topicmodel2016.wordpress.com/2016/06/15/topic-modelling-and-coloring-document-words/))
* RNN - нейронні мережі, для моделі мови на рівні окремих букв ([лінк](http://karpathy.github.io/2015/05/21/rnn-effectiveness/))
* ...


***

### Корисні посилання
* [теорія word2vec](https://en.wikipedia.org/wiki/Word2vec)
* [gensim - одна із найкращих бібліотек для маш. аналізу текстів, Python](https://radimrehurek.com/gensim/)
* [приклад застосування для продажу одягу](http://multithreaded.stitchfix.com/blog/2015/03/11/word-is-worth-a-thousand-vectors/)


### Приклад: схожі слова, зайві слова і пошук аналогій

In [1]:
import gensim
model = gensim.models.Word2Vec.load('models/wiki.uk.word2vec.model')


In [222]:
model[u'пиво'][1:40]

array([ 0.05304321,  0.00810462,  0.03724004,  0.10954185, -0.04059952,
        0.01651431,  0.02306625, -0.11219395,  0.00517311, -0.04896003,
       -0.04635075,  0.05704347,  0.00164528,  0.09318107, -0.03578118,
       -0.04156222,  0.0063322 ,  0.101604  ,  0.03367821,  0.02098054,
       -0.03198261,  0.03303163, -0.06150306, -0.02236838,  0.0303427 ,
        0.01784237, -0.00490326,  0.00359833, -0.06150605,  0.01166077,
        0.08295213, -0.05766207,  0.04183115,  0.01489175, -0.05265984,
        0.00902353,  0.02851386, -0.00856212, -0.05270547], dtype=float32)

#### Схожі слова

In [232]:
r = model.most_similar(u'вівторок')
for my_item in r:
    print my_item[0]

#r = model.most_similar(positive=[u'сила', u'голова'])
#for my_item in r:
#    print my_item[0]


понеділок
четвер
пъятницю
середу
пъятниця
ятниця
субота
суботу
неділя
неділю


#### Зайві слова

In [231]:
print model.doesnt_match(u"пиво горілка вино кефір".split())

кефір


In [230]:
print model.doesnt_match(u"пиво горілка вино хліб".split())

хліб


In [229]:
print model.similarity(u'пиво', u'вино')
print model.similarity(u'пиво', u'кефір')

0.725214369117
0.492852439011


#### Аналогії

In [240]:
list = [
[u'жінка', u'король', u'чоловік'],
[u'сука', u'король', u'кобель'],
[u'жінка', u'дядько', u'чоловік'],
[u'жінка', u'син', u'чоловік'],
[u'жінка', u'паротяг', u'чоловік'],
[u'красивий', u'ніякий', u'негарний'],
[u'сміливий', u'спокійний', u'боязкий'],
[u'хоробрий', u'людина', u'спокійний'],
[u'хоробрість', u'людина', u'страх'],
[u'хоробрість', u'жінка', u'страх'],
[u'жінка', u'син', u'дівчина'],
[u'зима', u'дощ', u'літо']]

for l in list:
    w,s = model.most_similar(positive=[l[0], l[1]],  negative=[l[2]], topn=5)[0]
    print l[0] + " - " + l[2] + " + " + l[1] + " = " + w
    

жінка - чоловік + король = королева
сука - кобель + король = королева
жінка - чоловік + дядько = тітка
жінка - чоловік + син = дочка
жінка - чоловік + паротяг = дрезина
красивий - негарний + ніякий = дивовижний
сміливий - боязкий + спокійний = рішучий
хоробрий - спокійний + людина = жінка
хоробрість - страх + людина = доблесть
хоробрість - страх + жінка = шабля
жінка - дівчина + син = онук
зима - літо + дощ = сніг


#### Кластеризація 

In [192]:
# Завантажуємро три списки слів на три теми - їжа, родина, мистецтво
# Вибираємо з моделі вектори для слів з цих списків

import numpy as np

with open('lang/food.txt', 'r') as infile:
    food_words = infile.readlines()

with open('lang/family.txt', 'r') as infile:
    family_words = infile.readlines()

with open('lang/art.txt', 'r') as infile:
    art_words = infile.readlines()

def getWordVecs(words):
    vecs = []
    new_words = []
    for word in words:
        word = word.replace('\n', '').decode(encoding='UTF-8')
        #print word
        try:
            vecs.append(model[word])
            new_words.append(word)
            #print model[word]
        except KeyError:
            continue
    #vecs = np.concatenate(vecs)
    #print model[u'анчоус']
    return new_words, np.array(vecs, dtype='float') #TSNE expects float type values



food_words, food_vecs = getWordVecs(food_words)
family_words, family_vecs = getWordVecs(family_words)
art_words, art_vecs = getWordVecs(art_words)




In [196]:
# Робимо tSNE 
# Задаємо колір груп


%matplotlib inline 

#from tsne import bh_sne
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

ts = TSNE(2)
reduced_vecs = ts.fit_transform(np.concatenate((food_vecs, family_vecs, art_vecs)))
print np.shape(reduced_vecs)

#plt.figure(figsize=(20,10))

#color points by word group to see if Word2Vec can separate them
colors = []
for i in range(len(reduced_vecs)):
    if i < len(food_vecs):
        #food words colored blue
        color = 'b'
    elif i >= len(food_vecs) and i < (len(food_vecs) + len(family_vecs)):
        #family words colored red
        color = 'r'
    else:
        #art words colored green
        color = 'g'
    colors.append(color)    
    #plt.plot(reduced_vecs[i,0], reduced_vecs[i,1], marker='o', color=color, markersize=10, alpha=0.3)

    
    



(660, 2)


In [218]:
# Інтерактивний графік

import mpld3
words = np.concatenate((food_words, family_words, art_words))
fig, ax = plt.subplots(subplot_kw=dict(axisbg='#EEEEEE'), figsize=(10,10))
css = "h3{background-color: white;}"

scatter = ax.scatter(reduced_vecs[:,0],
                     reduced_vecs[:,1],
                     c=colors,
                     s=34,
                     alpha=0.5,
                     cmap=plt.cm.jet)
ax.grid(color='white', linestyle='solid')

ax.set_title(u"Їжа, родина та мистецтво", size=40)

labels = ['<h3>{0}</h3>'.format(w.encode(encoding='UTF-8')) for w in words]
tooltip = mpld3.plugins.PointHTMLTooltip(scatter, labels=labels, css=css)
mpld3.plugins.connect(fig, tooltip)



mpld3.display()


### 50 тисяч випадкових слів української мови

![50000 випадкових слів](illustrations/50000_words.png)

### StitchFix

![до вагітності](illustrations/vectors_image1.png)
```python
model.most_similar('ITEM_3469', 'pregnant')
matches = list(filter(lambda x: 'ITEM_' in x[0], matches))
# ['ITEM_13792',
# 'ITEM_11275',
# 'ITEM_11868']
```


![до вагітності](illustrations/vectors_images_preg.png)